In [1]:
import pandas as pd
import folium
import folium.plugins as plugins
import numpy as np
import math

In [9]:
sz_car = pd.read_csv("../../data/travel_times_super_zones_driving.csv")
sz_non_car = pd.read_csv("../../data/travel_times_super_zones_non_car.csv")
zone_data = pd.read_csv("../../data/zone_coordinates_with_super_zones.csv")

In [4]:
def get_geojson_points(zone_data):

    all_points = []

    for index, row in zone_data.iterrows():
        
        # Define dimensions of box in grid (lon, lat) altså bortover, oppover
        if not pd.isnull(row['Super_center']):
            coordinates = [float(row['Super_center'].split(',')[0][1:]), float(row['Super_center'].split(',')[1][1:-1])]


            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            #"lower_left": lower_left,
                            #"upper_right": upper_right,
                        },
                        "features":[],
                        "Number": float(row['Super_center_nr'])}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Point",
                    "coordinates": coordinates,
                }
            }

            geo_json["features"].append(grid_feature)

            all_points.append(geo_json)

    return all_points

In [5]:
def get_geojson_super_zone_frames(zone_data):

    all_boxes = []

    for index, row in zone_data.iterrows():
        
        center = [float(row['Super_center'].split(',')[0][1:]), float(row['Super_center'].split(',')[1][1:-1])]
        
        # Define dimensions of box in grid (lon, lat) altså bortover, oppover
        if int(row['Super_center_nr']) < 48:
            
            # Define dimensions of box in grid (lon, lat) altså bortover, oppover
            lower_left = [center[0]-0.008734, center[1]-0.004537]
            upper_right = [center[0]+0.008734, center[1]+0.004537]
            upper_left = [center[0]-0.008734, center[1]+0.004537]
            lower_right = [center[0]+0.008734, center[1]-0.004537]
        
        else:
           
            # Define dimensions of box in grid (lon, lat) altså bortover, oppover
            lower_left = [center[0]-0.004367, center[1]-0.002268]
            upper_right = [center[0]+0.004367, center[1]+0.002268]
            upper_left = [center[0]-0.004367, center[1]+0.002268]
            lower_right = [center[0]+0.004367, center[1]-0.002268]
            
            
            
        # Define json coordinates for polygon
        coordinates = [
            upper_left,
            upper_right,
            lower_right,
            lower_left,
            upper_left
        ]

        geo_json = {"type": "FeatureCollection",
                    "properties":{
                        "lower_left": lower_left,
                        "upper_right": upper_right,
                    },
                    "features":[],
                    "Number": float(row['Super_center_nr'])}

        grid_feature = {
            "type":"Feature",
            "geometry":{
                "type":"Polygon",
                "coordinates": [coordinates],
            }
        }

        geo_json["features"].append(grid_feature)

        all_boxes.append(geo_json)

    return all_boxes

In [6]:
def get_geojson_grid(zone_data):
    
    all_boxes = []

    for index, row in zone_data.iterrows():
        
        # Define dimensions of box in grid (lon, lat) altså bortover, oppover
        lower_left = [float(row['Lower_left'].split(',')[0][1:]), float(row['Lower_left'].split(',')[1][1:-1])]
        upper_right = [float(row['Upper_right'].split(',')[0][1:]), float(row['Upper_right'].split(',')[1][1:-1])]
        upper_left = [lower_left[0], upper_right[1]]
        lower_right = [upper_right[0], lower_left[1]]
     
        # Define json coordinates for polygon
        coordinates = [
            upper_left,
            upper_right,
            lower_right,
            lower_left,
            upper_left
        ]

        geo_json = {"type": "FeatureCollection",
                    "properties":{
                        "lower_left": lower_left,
                        "upper_right": upper_right,
                    },
                    "features":[],
                    "Number": float(row['Zone'])}

        grid_feature = {
            "type":"Feature",
            "geometry":{
                "type":"Polygon",
                "coordinates": [coordinates],
            }
        }

        geo_json["features"].append(grid_feature)

        all_boxes.append(geo_json)

    return all_boxes
    

In [7]:
def plot_samples_on_grid(zone_data):
    
    lower_left = [59.855331, 10.601628]
    upper_right = [59.973287, 10.950989]
    
    center = [lower_left[0]+(upper_right[0]-lower_left[0])/2,lower_left[1]+(upper_right[1]-lower_left[1])/2]

    grid_size = (26,40)
    
    map = folium.Map(center, zoom_start = 10.8)
    
    grid = get_geojson_grid(zone_data)
    points = get_geojson_points(zone_data)
    squares = get_geojson_super_zone_frames(zone_data)
    
    #Grid
    
    for i, geo_json in enumerate(grid):
        
        gj = folium.GeoJson(geo_json,
                style_function=lambda feature: {#'fillColor': color,
                                                'color':'#df80e8',
                                                'weight': 2,
                                                'dashArray': '5, 5',
                                                'fillOpacity': 0.3}).add_to(map)
        popup_string = "Zone "+ str(int(geo_json['Number']))
        
        gj.add_child(folium.Popup(popup_string, max_width=400))
    
    #Points superzones
    '''
    for i, geo_json in enumerate(points):
        
        gj = folium.GeoJson(geo_json,
                style_function=lambda feature: {#'fillColor': color,
                                                'color':'#df80e8',
                                                'weight': 5,
                                                'dashArray': '5, 5',
                                                'fillOpacity': 0.3}).add_to(map)
        popup_string = "Zone "+ str(int(geo_json['Number']))
        
        gj.add_child(folium.Popup(popup_string, max_width=400))
    '''
    
    #squares superzones
    for i, geo_json in enumerate(squares):
        
        gj = folium.GeoJson(geo_json,
                style_function=lambda feature: {#'fillColor': color,
                                                'color':'#32a834',
                                                'weight': 5,
                                                'dashArray': '5, 5',
                                                'fillOpacity': 0.0}).add_to(map)
        popup_string = "Zone "+ str(int(geo_json['Number']))
        
        gj.add_child(folium.Popup(popup_string, max_width=400))

    display(map)

In [11]:
plot_samples_on_grid(zone_data)

In [10]:
zone_data.head()

,Unnamed: 0,Zone,Lower_left,Upper_right,Center,Super_center,Super_center_nr
0,0,1,"[10.793777, 59.964213]","[10.802511, 59.96875]","[10.798144, 59.966482]","[10.793777, 59.964213]",1.0
1,1,2,"[10.785043, 59.964213]","[10.793777, 59.96875]","[10.78941, 59.966482]","[10.793777, 59.964213]",1.0
2,2,3,"[10.776308, 59.964213]","[10.785043, 59.96875]","[10.780676, 59.966482]","[10.780676, 59.966482]",48.0
3,3,4,"[10.767574, 59.964213]","[10.776309, 59.96875]","[10.771941, 59.966482]","[10.771941, 59.966482]",49.0
4,4,5,"[10.75884, 59.964213]","[10.767574, 59.96875]","[10.763207, 59.966482]","[10.75884, 59.964213]",2.0


In [12]:
sz_car.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,1,0,592,789,333,608,779,972,417,366,...,1008,1169,1246,1195,1379,1102,1273,1187,1363,1332
1,2,588,0,405,646,338,508,730,761,481,...,973,1135,1211,1160,1344,1067,1238,1152,1328,1297
2,3,801,421,0,860,413,315,579,820,572,...,1064,1225,1302,1251,1435,1158,1329,1243,1419,1388
3,4,318,648,765,0,516,642,859,289,253,...,895,1056,1133,1082,1266,989,1160,1074,1250,1219
4,5,629,353,403,596,0,280,557,564,316,...,808,969,1046,995,1178,902,1073,986,1163,1131


In [13]:
travel_times = np.array([254,254])